<a href="https://colab.research.google.com/github/ScriptsRemote/SCRIPTSREMOTE/blob/main/00_NDWI_AMBGEO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##instalação das bibliotecas
! pip install geemap earthengine-api

In [ ]:
#importar bibliotecas
import geemap
import ee

In [ ]:
#Autenticação
ee.Authenticate()
ee.Initialize()

In [ ]:
##Definir a nossa região de interesse
roi = ee.Geometry.Point([-54.90027550627894, -30.253775512590703])

##Definir nossa coleção de imagens
collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
                          .filterBounds(roi)\
                          .filterDate('2013-01-01','2023-01-01')\
                          .filter(ee.Filter.lt('CLOUD_COVER',1))

print('Qtds',collection.size().getInfo())

Qtds 38


In [ ]:
##Criar uma função para calcular o NDWI
def ndwi(img):
  ##Fator de Escala bandas opticas
  opticalBands = img.select('SR_B.').multiply(0.0000275).add(-0.2)
  ##Fator de Escala bandas termais
  thermalBands = img.select('ST_B.*').multiply(0.00341802).add(149.0)
  ##NDWI
  Ndwi_image = img.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
  ##Aplicando e adicionando as bandas com fator de escala
  return img.addBands(opticalBands, None, True).addBands(thermalBands, None, True)\
         .addBands(Ndwi_image).copyProperties(img, ["system:time_start"])\
         .set('date', img.date().format('YYYY-MM-dd'))

In [ ]:
##Aplicando fator de escala e NDWI em todas as imagens
collection_ndwi= collection.map(ndwi)
print('Qtds imagens',collection_ndwi.size().getInfo())

Qtds imagens 38


In [ ]:
###Criar mascaras para identificar valor máximos e médios
mask_max = collection_ndwi.select('NDWI').max().gt(0).selfMask()
mask_mean = collection_ndwi.select('NDWI').mean().gt(0).selfMask()

In [ ]:
# Defina um método para exibir blocos de imagens do Earth Engine em um folium map
Map = geemap.Map()
##Centralizando a imagem
Map = geemap.Map(location=[-30.2, -54.93], zoom_start= 10)
#Para ver uma visualização de satélite do Google como um mapa base
# Map.add_basemap('HYBRID')
Map.add_ee_layer(collection_ndwi.median(), {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
                                            'min': 0.03,'max': 0.19}, 'RGB')
##Adicionando Layer com as máscaras
Map.add_ee_layer(mask_max,{'palette':['blue'],'min':0, 'max':1},'NDWI max')
Map.add_ee_layer(mask_mean,{'palette':['cyan'],'min':0, 'max':1},'NDWI')
display(Map)

Map(center=[-30.2, -54.93], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…